### Import libraries

In [ ]:
# import libraries
import pandas as pd
import numpy as np
from scipy.stats import linregress
from scipy.interpolate import make_smoothing_spline
import plotly.graph_objects as go
from PIL import Image

pd.set_option('display.max_columns', None)

### Get data

In [ ]:
# load data
data = pd.read_csv("LPD_2024_public.csv")

In [ ]:
# preview data
print(data.shape)
data.head()

In [ ]:
# set marine class weights
indexes = ["Fishes", "Birds", "Mammals", "Reptiles"]
weights_class = pd.DataFrame(index=indexes)
weights_class["Arctic"] = [0.754, 0.205, 0.041, 0.000]
weights_class["Atlantic north temperate"] = [0.833, 0.144, 0.021, 0.003]
weights_class["Atlantic tropical and subtropical"] = [0.895, 0.094, 0.008, 0.002]
weights_class["Pacific north temperate"] = [0.880, 0.090, 0.028, 0.001]
weights_class["Tropical and subtropical Indo-Pacific"] = [0.932, 0.056, 0.006, 0.006]
weights_class["South temperate and Antarctic"] = [0.918, 0.056, 0.024, 0.002]

# set marine realm weights
columns = ["Arctic", "Atlantic north temperate", "Atlantic tropical and subtropical", "Pacific north temperate",
    "Tropical and subtropical Indo-Pacific", "South temperate and Antarctic"]
weights_realm = pd.DataFrame(index=["All classes"], columns = columns)
weights_realm.loc["All classes", :] = [0.02, 0.09, 0.20, 0.08, 0.50, 0.11]


### Select relevant data

In [ ]:
# rename relevant classes
mask_aves = data["Class"] == "Aves"
data.loc[mask_aves, "Class"] = "Birds"
mask_mammalia = data["Class"] == "Mammalia"
data.loc[mask_mammalia, "Class"] = "Mammals"
mask_reptilia = data["Class"] == "Reptilia"
data.loc[mask_reptilia, "Class"] = "Reptiles"
mask_actinopteri = data["Class"] == "Actinopteri" # fish
mask_elasmobranchii = data["Class"] == "Elasmobranchii" # fish
mask_holocephali = data["Class"] == "Holocephali" # fish
mask_petomyzonti = data["Class"] == "Petromyzonti" # fish
mask_coelacanthi = data["Class"] == "Coelacanthi" # fish
mask_myxini = data["Class"] == "Myxini" # kind of fishy worm
mask_fishes = mask_actinopteri | mask_elasmobranchii | mask_holocephali | \
    mask_petomyzonti | mask_coelacanthi | mask_myxini
data.loc[mask_fishes, "Class"] = "Fishes"

In [ ]:
# select relevant data
mask_included = data["Included in LPR2024"] == 1
mask_replicate = data["Replicate"] == 0
mask_native = data["Native"] == 1
mask_system = data["System"] == "Marine"
mask_all = mask_included & mask_replicate & mask_native & mask_system
data = data.loc[mask_all, :].reset_index(drop=True)
print(data.shape)

### Clean data

In [ ]:
# get year columns
year_columns = np.arange(1950, 2021, 1).astype(str)

In [ ]:
# check for data containing only zeros
for i in range(data.shape[0]):
    mask_notnull = data.loc[i, year_columns].notnull()
    mask_zeros = data.loc[i, year_columns] == 0
    if mask_notnull.sum() == mask_zeros.sum():
       index_zeros = mask_zeros[mask_zeros == True].index
       data.loc[i, index_zeros] = 1

In [ ]:
# check for data containing sparse zeros
for i in range(data.shape[0]):
    mask_zeros = data.loc[i, year_columns] == 0
    if mask_zeros.sum() > 0:
       ymean = data.loc[i, year_columns].mean()
       data.loc[i, year_columns] = data.loc[i, year_columns] + (ymean / 100)

### Intepolate missing values

In [ ]:
# copy data
data_interp = data.copy()

In [ ]:
# interpolate missing values
for i in range(data_interp.shape[0]):
    # get leading and tailing x values
    mask_notnull = data_interp.loc[i, year_columns].notnull()
    first_ok = mask_notnull[mask_notnull == True].index[0]
    last_ok = mask_notnull[mask_notnull == True].index[-1]
    # check for the presence of nans
    data_current = data_interp.loc[i, first_ok:last_ok]
    mask_nan = data_current.isnull()
    # skip if no interpolation needed
    if mask_nan.sum() == 0:
        continue
    # get input data
    data_input = data_interp.loc[i, first_ok:last_ok]
    mask_input = data_interp.loc[i, first_ok:last_ok].notnull()
    x = data_input.index[mask_input].astype(float)
    y = data_input.values[mask_input].astype(float)
    ylog = np.log10(y)
    xnew_columns = data_input.index
    xnew_values = xnew_columns.astype(float)
    # case with linear regression
    if mask_notnull.sum() < 6:
        result = linregress(x, ylog)
        ynew = result.slope * xnew_values + result.intercept
        data_interp.loc[i, xnew_columns] = ynew
    # case with smoothing spline
    else:
        spl = make_smoothing_spline(x, ylog, lam=5)
        ynew = spl(xnew_values)
        data_interp.loc[i, xnew_columns] = ynew


In [ ]:
# plot available records per year
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=year_columns,
        y=data.loc[:, year_columns].notnull().sum(),
        name="data"
    )
)
fig.add_trace(
    go.Scatter(
        x=year_columns,
        y=data_interp.loc[:, year_columns].notnull().sum(),
        name="data_interp"
    )
)
fig.show()

### Get trends

In [ ]:
# copy data before getting trends
data_trends = data_interp.copy()
data_trends.loc[:, year_columns] = np.nan

In [ ]:
# get trends
for i in range(data_trends.shape[0]):
    # get leading and tailing x values
    mask_notnull = data_interp.loc[i, year_columns].notnull()
    first_ok = mask_notnull[mask_notnull == True].index[0]
    second_ok = mask_notnull[mask_notnull == True].index[1]
    last_ok = mask_notnull[mask_notnull == True].index[-1]
    # get trends
    y = data_interp.loc[i, first_ok:last_ok].values.astype(float)
    trend = y[1:] - y[:-1]
    data_trends.loc[i, second_ok:last_ok] = trend

In [ ]:
# cap outliers to 10-fold increase or decrease
for column in year_columns:
    mask_low = data_trends.loc[:, column] < -1
    mask_high = data_trends.loc[:, column] > 1
    data_trends.loc[mask_low, column] = -1
    data_trends.loc[mask_high, column] = 1

### Bootstrap LPI

In [ ]:
repeat_nb = 1000
sample_size = 500

year_columns = np.arange(1970, 2021, 1).astype(str)
np.random.seed(1234)

lpi_all = pd.DataFrame(index=np.arange(0, repeat_nb, 1), columns=year_columns)

for i in range(0, repeat_nb):
    print(i)

    # copy data and get sample
    data_sample = data_trends.copy()
    data_sample = data_sample.sample(sample_size, replace=False)

    # average trends for each species realm wise and ocean wise
    grouping_columns = ["M_realm", "M_ocean", "Class", "Binomial"]
    data_species_ocean = data_sample.groupby(grouping_columns)[year_columns].mean().reset_index()

    # average trends for each species realm wise
    grouping_columns = ["M_realm", "Class", "Binomial"]
    data_species_realm = data_species_ocean.groupby(grouping_columns)[year_columns].mean().reset_index()

    # average trends for each class realm wise
    grouping_columns = ["M_realm", "Class"]
    data_class_realm = data_species_realm.groupby(grouping_columns)[year_columns].mean().reset_index()
    
    # apply weights for each class of each realm
    realms = data_class_realm["M_realm"].unique()
    classes = data_class_realm["Class"].unique()
    for realm in realms:
        for classe in classes:
            mask_realm = data_class_realm["M_realm"] == realm
            mask_class = data_class_realm["Class"] == classe
            mask_all = mask_realm & mask_class
            if mask_all.sum() == 0:
                continue
            trends = data_class_realm.loc[mask_all, year_columns]
            weight = weights_class.loc[classe, realm]
            data_class_realm.loc[mask_all, year_columns] = trends * weight

    # average trends for each realm
    grouping_columns = ["M_realm"]
    data_realm = data_class_realm.groupby(grouping_columns)[year_columns].mean().reset_index()
    
    # apply weights for each realm
    realms = data_realm["M_realm"].unique()
    for realm in realms:
        mask_realm = data_realm["M_realm"] == realm
        trends = data_realm.loc[mask_realm, year_columns]
        weight = weights_realm.loc["All classes", realm]
        data_realm.loc[mask_realm, year_columns] = trends * weight

    # get global trend
    global_trend = data_realm[year_columns].mean()

    # get global index
    global_index = global_trend.copy()
    global_index[year_columns] = np.nan
    global_index["1970"] = 1
    for j in range(len(year_columns) - 1):
        index_previous = global_index[year_columns[j]]
        mean_trend = global_trend[year_columns[j+1]]
        global_index[year_columns[j+1]] = index_previous * (10 ** mean_trend)

    # store indexes
    lpi_all.loc[i, :] = global_index[year_columns].values


### Plot LPI mean and 95% confidence intervals

In [ ]:
# get mean
lpi_mean = np.round(lpi_all.mean() * 100, 1)

# get confidence intervals (99%, z=2.58)
lpi_error = 100 * 2.58 * lpi_all.std() / np.sqrt(repeat_nb)
lpi_upper = list((lpi_mean + lpi_error).values)
lpi_lower = list((lpi_mean - lpi_error).values)

# set years
year_columns = list(np.arange(1970, 2021, 1))

In [ ]:
# plot lpi
fig = go.Figure()

# add wave
image_size = 120
fig.add_layout_image(
    dict(
        source=Image.open("Vague Blutopia-Tofu.png"),
        xref="x",
        yref="y",
        xanchor="center",
        yanchor="middle",
        x=1995,
        y=60,
        sizex=image_size,
        sizey=image_size,
        sizing="contain",
        opacity=1.0,
        layer="below",
        visible=True
    ),
)

# plot confidence intervals
fig.add_trace(
    go.Scatter(
        x=year_columns+year_columns[::-1],
        y=lpi_upper+lpi_lower[::-1],
        fill="toself",
        fillcolor="#107069",
        line=dict(color="rgba(255,255,255,0)"),
        hoverinfo="skip"
    )
)

# plot mean
fig.add_trace(
    go.Scatter(
        x=year_columns,
        y=lpi_mean,
        line=dict(
            color="#FDF2EE",
            width=1),
        mode="lines",
        customdata=lpi_mean,
        hovertemplate="<b>IPV = %{customdata}</b><extra></extra>"
    )
)

fig.update_xaxes(
    showgrid=False,
    tickvals=np.arange(1970, 2025, 5),
    ticklabelstandoff=10,
)

fig.update_yaxes(
    title=dict(
        text="Indice Planète Vivante (1970 = 100%)",
        font_family="Montserrat",
        font=dict(size=16)
    ),
    range=[10, 110],
    tickvals=np.arange(0, 120, 20),
    ticklabelstandoff=20,
    gridcolor="#58B6AF"
)

fig.update_layout(
    showlegend=False,
    plot_bgcolor="white",
    font_color="#58B6AF",
    font_family="Montserrat",
    hoverlabel=dict(
        font_size=14,
        bordercolor="#107069",
        bgcolor="#FDF2EE"
    ),
    margin=dict(
        l=20,
        r=20,
        t=20,
        b=20
    ),
    height=400,
    width=550,

)

fig.show()

In [ ]:
fig.write_html("lpi.html", include_plotlyjs="cdn")